<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Изучение-данных" data-toc-modified-id="Изучение-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Изучение данных</a></span></li><li><span><a href="#Предобработка-данных" data-toc-modified-id="Предобработка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Предобработка данных</a></span></li><li><span><a href="#Подготовка-признаков" data-toc-modified-id="Подготовка-признаков-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Подготовка признаков</a></span></li><li><span><a href="#Обучение-без-учёта-дисбаланса-классов" data-toc-modified-id="Обучение-без-учёта-дисбаланса-классов-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Обучение без учёта дисбаланса классов</a></span><ul class="toc-item"><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Случайный лес</a></span></li><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li></ul></li><li><span><a href="#Обучение-с-учётом-дисбаланса" data-toc-modified-id="Обучение-с-учётом-дисбаланса-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Обучение с учётом дисбаланса</a></span><ul class="toc-item"><li><span><a href="#class_weight='balanced'" data-toc-modified-id="class_weight='balanced'-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span><code>class_weight='balanced'</code></a></span><ul class="toc-item"><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-5.1.1"><span class="toc-item-num">5.1.1&nbsp;&nbsp;</span>Случайный лес</a></span></li><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-5.1.2"><span class="toc-item-num">5.1.2&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li></ul></li><li><span><a href="#Upsampling-&amp;-downsampling" data-toc-modified-id="Upsampling-&amp;-downsampling-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Upsampling &amp; downsampling</a></span><ul class="toc-item"><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-5.2.1"><span class="toc-item-num">5.2.1&nbsp;&nbsp;</span>Случайный лес</a></span></li><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-5.2.2"><span class="toc-item-num">5.2.2&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Метод-k-ближайших-соседей" data-toc-modified-id="Метод-k-ближайших-соседей-5.2.3"><span class="toc-item-num">5.2.3&nbsp;&nbsp;</span>Метод k ближайших соседей</a></span></li></ul></li></ul></li><li><span><a href="#Тестирование-моделей" data-toc-modified-id="Тестирование-моделей-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Тестирование моделей</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Вывод</a></span></li></ul></div>

# Отток клиентов

**Цель**: спрогнозировать, уйдёт клиент из банка в ближайшее время или нет.

У нас имеются исторические данные о поведении клиентов и расторжении договоров с банком. Построим модель с предельно большим значением F1-меры (нужно довести метрику до 0.59). Дополнительно измерим AUC-ROC и сравним её значение с F1-мерой.

Данные находятся в файле `Churn.csv`

**План работ**
1. Загрузим и подготовим данные. Поясним порядок действий.
2. Исследуйем баланс классов, обучим модель без учёта дисбаланса. Кратко опишем выводы.
3. Улучшим качество модели, учитывая дисбаланс классов. Обучим разные модели и найдем лучшую. Кратко опишем выводы.
4. Проведем финальное тестирование.

## Изучение данных

In [1]:
#!pip install -Iv scikit-learn==0.22 --user

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import seaborn as sns
import os
#from catboost import CatBoostClassifier, Pool
import sklearn as sk
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.utils import shuffle
import warnings
#from random import seed, random
#import statistics

In [3]:
dt.datetime.today().strftime("%d.%m.%Y %H:%M")

'09.06.2022 13:53'

In [4]:
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [5]:
pth1 = '/datasets/Churn.csv'
pth2 = 'Churn.csv'
if os.path.exists(pth1):
    df = pd.read_csv(pth1)
elif os.path.exists(pth2):
    df = pd.read_csv(pth2)
else:
    print('Something is wrong')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             9091 non-null float64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [7]:
df.head(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


В таблице 14 столбцов, 10000 строк. Пропуски есть только в столбце `tenure`.

Согласно документации:

`RowNumber` — индекс строки в данных;
<br>`CustomerId` — уникальный идентификатор клиента;
<br>`Surname` — фамилия;
<br>`CreditScore` — кредитный рейтинг;
<br>`Geography` — страна проживания;
<br>`Gender` — пол;
<br>`Age` — возраст;
<br>`Tenure` — сколько лет человек является клиентом банка;
<br>`Balance` — баланс на счёте;
<br>`NumOfProducts` — количество продуктов банка, используемых клиентом;
<br>`HasCrCard` — наличие кредитной карты;
<br>`IsActiveMember` — активность клиента;
<br>`EstimatedSalary` — предполагаемая зарплата;

`Exited` — факт ухода клиента (целевой признак).

## Предобработка данных

Столбцы именованы в стиле `CamelCase`, приведём к `snake_case`.

In [8]:
df.columns = [ 'row_number', 'customer_id', 'surname', 'credit_score', 'geography', 'gender', 'age', 'tenure', 
              'balance', 'num_of_products', 'has_credit_card', 'is_active_member', 'estimated_salary', 'exited' ]
df.columns

Index(['row_number', 'customer_id', 'surname', 'credit_score', 'geography',
       'gender', 'age', 'tenure', 'balance', 'num_of_products',
       'has_credit_card', 'is_active_member', 'estimated_salary', 'exited'],
      dtype='object')

Проверим пропуски в `tenure`.

In [9]:
df['tenure'].value_counts(dropna=False)

1.0     952
2.0     950
8.0     933
3.0     928
5.0     927
7.0     925
NaN     909
4.0     885
9.0     882
6.0     881
10.0    446
0.0     382
Name: tenure, dtype: int64

Это почти 10% данных, поэтому удалять их не хотелось бы. Нужно заполнить пропуски.

*Не смогла использовать алгоритм заполнения по k ближайшим соседям `KNNImputer`, т.к. он появился в sklearn v0.22, а в окружении Практикума указана версия 0.21.2. Пробовала обновить версию, но не получилось. (Anaconda)*

In [10]:
#imputer = imp.KNNImputer(n_neighbors=2, weights="uniform")
#df['tenure'] = imputer.fit_transform(df[['tenure']])
sk.__version__

'0.21.2'

<div class="alert alert-block alert-success">
<b>Комментарий ревьюера👍:</b> Из-за того, что в хабе предустановлены библиотеки, то к сожалению бывают такие косяки =( Изменить версию не получится
</div>

Используем `IterativeImputer`.

In [11]:
numeric = [ 'credit_score', 'age', 'tenure', 'balance', 'num_of_products', 'estimated_salary'
          , 'has_credit_card', 'is_active_member' ]
imputer = IterativeImputer()
df[numeric] = imputer.fit_transform(df[numeric])
df['tenure'].value_counts(dropna=False)

1.000000    952
2.000000    950
8.000000    933
3.000000    928
5.000000    927
           ... 
5.032477      1
5.041131      1
5.028779      1
4.969563      1
5.001389      1
Name: tenure, Length: 920, dtype: int64

Проверим дубли.

In [12]:
df.duplicated().sum()

0

Столбцы `row_number`, `customer_id`, `surname` нельзя/нецелесообразно использовать для обучения модели, удалим их.

In [13]:
df = df.drop(columns=[  'row_number', 'customer_id', 'surname'])

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
credit_score        10000 non-null float64
geography           10000 non-null object
gender              10000 non-null object
age                 10000 non-null float64
tenure              10000 non-null float64
balance             10000 non-null float64
num_of_products     10000 non-null float64
has_credit_card     10000 non-null float64
is_active_member    10000 non-null float64
estimated_salary    10000 non-null float64
exited              10000 non-null int64
dtypes: float64(8), int64(1), object(2)
memory usage: 859.5+ KB


Остаётся 10 признаков и 1 целевой признак.

Проверим данные в остальных столбцах.

In [15]:
df.credit_score.describe()

count    10000.000000
mean       650.528800
std         96.653299
min        350.000000
25%        584.000000
50%        652.000000
75%        718.000000
max        850.000000
Name: credit_score, dtype: float64

In [16]:
df.geography.value_counts()

France     5014
Germany    2509
Spain      2477
Name: geography, dtype: int64

In [17]:
df.gender.value_counts()

Male      5457
Female    4543
Name: gender, dtype: int64

In [18]:
df.age.describe()

count    10000.000000
mean        38.921800
std         10.487806
min         18.000000
25%         32.000000
50%         37.000000
75%         44.000000
max         92.000000
Name: age, dtype: float64

In [19]:
df.balance.describe()

count     10000.000000
mean      76485.889288
std       62397.405202
min           0.000000
25%           0.000000
50%       97198.540000
75%      127644.240000
max      250898.090000
Name: balance, dtype: float64

In [20]:
df.num_of_products.value_counts()

1.0    5084
2.0    4590
3.0     266
4.0      60
Name: num_of_products, dtype: int64

In [21]:
df.has_credit_card.value_counts()

1.0    7055
0.0    2945
Name: has_credit_card, dtype: int64

In [22]:
df.is_active_member.value_counts()

1.0    5151
0.0    4849
Name: is_active_member, dtype: int64

In [23]:
df.estimated_salary.describe()

count     10000.000000
mean     100090.239881
std       57510.492818
min          11.580000
25%       51002.110000
50%      100193.915000
75%      149388.247500
max      199992.480000
Name: estimated_salary, dtype: float64

In [24]:
df.exited.value_counts()

0    7963
1    2037
Name: exited, dtype: int64

Остальные данные выглядят адекватно, никакая больше предобработка не требуется.

## Подготовка признаков

2 признака: `geography`, `gender` - нужно перевести в числовой вид. Воспользуемся техникой *One-Hot Encoding*, т.к. она работает со всеми моделями, и разных значений в этих признаках не так много, поэтому не будет слишком много дополнительных столбцов.

In [25]:
df_ohe = pd.get_dummies(df, drop_first=True)
df_ohe.head(2)

,credit_score,age,tenure,balance,num_of_products,has_credit_card,is_active_member,estimated_salary,exited,geography_Germany,geography_Spain,gender_Male
0,619.0,42.0,2.0,0.00,1.0,1.0,1.0,101348.88,1,0,0,0
1,608.0,41.0,1.0,83807.86,1.0,0.0,1.0,112542.58,0,0,1,0


Разделим выборку на признаки и целевой признак.

In [26]:
target = df_ohe['exited']
features = df_ohe.drop('exited', axis=1)

Масштабируем все (количественные) признаки, чтобы они были равнозначны для моделей.

In [27]:
numeric = [ 'credit_score', 'age', 'tenure', 'balance', 'num_of_products', 'estimated_salary' ]
#           , 'has_credit_card', 'is_active_member', 'geography_Germany', 'geography_Spain', 'gender_Male']

scaler = StandardScaler(with_std=True)
# scaler = MinMaxScaler()
scaler.fit(features[numeric])
features[numeric] = scaler.transform(features[numeric])

features.head(2)

,credit_score,age,tenure,balance,num_of_products,has_credit_card,is_active_member,estimated_salary,geography_Germany,geography_Spain,gender_Male
0,-0.326221,0.293517,-1.086151,-1.225848,-0.911583,1.0,1.0,0.021886,0,0,0
1,-0.440036,0.198164,-1.448483,0.117350,-0.911583,0.0,1.0,0.216534,0,1,0


Разделим выборку на обучающую, валидационную и тестовую.

In [28]:
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, 
    test_size=0.33333, 
    shuffle=True, stratify=target, 
    random_state=486745)

features_test, features_valid, target_test, target_valid = train_test_split(
    features_valid, target_valid, 
    test_size=0.5, 
    shuffle=True, stratify=target_valid, 
    random_state=154648)

print(features_train.shape) 
print(features_valid.shape) 
print(features_test.shape) 
print('\n', target_train.mean(), target_valid.mean(), target_test.mean(), sep='\t')

(6666, 11)
(1667, 11)
(1667, 11)

	0.20372037203720372	0.20335932813437313	0.20395920815836832


Подготовим также увеличенные тренировочные выборки, которые можно использовать для обучения (заново) после подбора гиперпараметров либо сразу в GridSearch.

In [29]:
features_train_2 = pd.concat([features_train, features_valid], ignore_index=True)
target_train_2 = pd.concat([target_train, target_valid], ignore_index=True)
print(features_train_2.shape, target_train_2.shape)

(8333, 11) (8333,)


## Обучение без учёта дисбаланса классов

Исследуем баланс классов, обучим модель без учёта дисбаланса. Кратко опишем выводы.

In [30]:
df.exited.value_counts() / df.exited.count()

0    0.7963
1    0.2037
Name: exited, dtype: float64

Есть дисбаланс целевых классов: **20 / 80**.

### Случайный лес

In [31]:
%%time
forest_model = RandomForestClassifier(random_state=12345)

params = dict(n_estimators=range(97,137,4), max_depth=range(14,19))
grid = GridSearchCV(forest_model, params, n_jobs=4, verbose=0, scoring='f1', cv=2)
grid.fit(features_train_2, target_train_2);

print(grid.best_params_)
best_forest = grid.best_estimator_
print(best_forest)

predicted_train_2 = best_forest.predict(features_train_2)
print('f1 на увеличенной (обучающе-валидационной) выборке:', f1_score(target_train_2, predicted_train_2))
# predicted_train = best_forest.predict(features_train)
# print('f1 на обучающей выборке:', f1_score(target_train, predicted_train))
# predicted_valid = best_forest.predict(features_valid)
# print('f1 на валидационной выборке:', f1_score(target_valid, predicted_valid))
predicted_test = best_forest.predict(features_test)
print('f1 на тестовой выборке:', f1_score(target_test, predicted_test))

{'max_depth': 17, 'n_estimators': 129}
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=17, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=129,
                       n_jobs=None, oob_score=False, random_state=12345,
                       verbose=0, warm_start=False)
f1 на увеличенной (обучающе-валидационной) выборке: 0.9871680095493882
f1 на тестовой выборке: 0.5608856088560886
Wall time: 30 s


In [32]:
probabilities_test = best_forest.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]
auc_roc = roc_auc_score(target_test, probabilities_one_test)
print('AUC-ROC на тестовой выборке:', auc_roc)

AUC-ROC на тестовой выборке: 0.8506117292433174


Без учёта дисбаланса классов пока не удалось преодолеть установленный для ***f1*** порог **0.59**.

### Логистическая регрессия

In [33]:
%%time
log_reg_model = LogisticRegression(random_state=1234567, max_iter=300, class_weight=None)

params = dict(C=[1,2,5,10], 
              penalty=['l2','l1'],
              solver=['liblinear','warn']) 
grid = GridSearchCV(log_reg_model, params, verbose=0, scoring='f1', cv=5)
grid.fit(features_train_2, target_train_2);

print(grid.best_params_)
best_log_reg_model = grid.best_estimator_
print(best_log_reg_model)

predicted_train_2 = best_log_reg_model.predict(features_train_2)
print('f1 на увеличенной (обучающе-валидационной) выборке:', f1_score(target_train_2, predicted_train_2))
# predicted_train = best_log_reg_model.predict(features_train)
# print('f1 на обучающей выборке:', f1_score(target_train, predicted_train))
# predicted_valid = best_log_reg_model.predict(features_valid)
# print("f1 на валидационной выборке:", f1_score(target_valid, predicted_valid))
predicted_test = best_log_reg_model.predict(features_test)
print('f1 на тестовой выборке:', f1_score(target_test, predicted_test))

{'C': 10, 'penalty': 'l2', 'solver': 'liblinear'}
LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=300,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=1234567, solver='liblinear', tol=0.0001,
                   verbose=0, warm_start=False)
f1 на увеличенной (обучающе-валидационной) выборке: 0.32224622030237576
f1 на тестовой выборке: 0.28633405639913234
Wall time: 3.73 s


In [34]:
probabilities_test = best_log_reg_model.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]
auc_roc = roc_auc_score(target_test, probabilities_one_test)
print('AUC-ROC на тестовой выборке:', auc_roc)

AUC-ROC на тестовой выборке: 0.7682787357595637


Низкий показатель.

## Обучение с учётом дисбаланса
Теперь будем учитывать дисбаланс классов. Обучим разные модели и выберем лучшую.

### `class_weight='balanced'`

#### Случайный лес

In [35]:
%%time
forest_model = RandomForestClassifier(random_state=12345, class_weight='balanced')

params = dict(n_estimators=range(111,139,2), max_depth=range(12,19))
grid = GridSearchCV(forest_model, params, n_jobs=4, verbose=0, scoring='f1', cv=2)
grid.fit(features_train_2, target_train_2);

print(grid.best_params_)
best_forest = grid.best_estimator_
print(best_forest)

predicted_train_2 = best_forest.predict(features_train_2)
print('f1 на увеличенной (обучающе-валидационной) выборке:', f1_score(target_train_2, predicted_train_2))
# predicted_train = best_forest.predict(features_train)
# print('f1 на обучающей выборке:', f1_score(target_train, predicted_train))
# predicted_valid = best_forest.predict(features_valid)
# print('f1 на валидационной выборке:', f1_score(target_valid, predicted_valid))
predicted_test = best_forest.predict(features_test)
print('f1 на тестовой выборке:', f1_score(target_test, predicted_test))

{'max_depth': 12, 'n_estimators': 135}
RandomForestClassifier(bootstrap=True, class_weight='balanced',
                       criterion='gini', max_depth=12, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=135, n_jobs=None, oob_score=False,
                       random_state=12345, verbose=0, warm_start=False)
f1 на увеличенной (обучающе-валидационной) выборке: 0.8854166666666667
f1 на тестовой выборке: 0.6104294478527607
Wall time: 2min


In [36]:
probabilities_test = best_forest.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]
auc_roc = roc_auc_score(target_test, probabilities_one_test)
print('AUC-ROC на тестовой выборке:', auc_roc)

AUC-ROC на тестовой выборке: 0.8499534553836605


Удалось получить существенный выигрыш по сравнению с предыдущим результатом случайного леса и преодолеть пороговое значение 0.59.

#### Логистическая регрессия

In [37]:
%%time
log_reg_model = LogisticRegression(random_state=1234567, max_iter=300, class_weight='balanced')

params = dict(C=[1,2,5,10], 
              penalty=['l2','l1'],
              solver=['liblinear','warn']
             ) 
grid = GridSearchCV(log_reg_model, params, verbose=0, scoring='f1', cv=2)
grid.fit(features_train_2, target_train_2);

print(grid.best_params_)
best_log_reg_model = grid.best_estimator_
print(best_log_reg_model)

predicted_train_2 = best_log_reg_model.predict(features_train_2)
print('f1 на увеличенной (обучающе-валидационной) выборке:', f1_score(target_train_2, predicted_train_2))
# predicted_train = best_log_reg_model.predict(features_train)
# print('f1 на обучающей выборке:', f1_score(target_train, predicted_train))
# predicted_valid = best_log_reg_model.predict(features_valid)
# print("f1 на валидационной выборке:", f1_score(target_valid, predicted_valid))
predicted_test = best_log_reg_model.predict(features_test)
print('f1 на тестовой выборке:', f1_score(target_test, predicted_test))

{'C': 2, 'penalty': 'l2', 'solver': 'liblinear'}
LogisticRegression(C=2, class_weight='balanced', dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=300,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=1234567, solver='liblinear', tol=0.0001,
                   verbose=0, warm_start=False)
f1 на увеличенной (обучающе-валидационной) выборке: 0.4974683544303797
f1 на тестовой выборке: 0.49438202247191015
Wall time: 1.34 s


In [38]:
probabilities_test = best_log_reg_model.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]
auc_roc = roc_auc_score(target_test, probabilities_one_test)
print('AUC-ROC на тестовой выборке:', auc_roc)

AUC-ROC на тестовой выборке: 0.7699011481005364


Уже существенный выигрыш по сравнению с результатом лог.регрессии без указания параметра `class_weight`, но f1 всё ещё очень далеко до 0.59.

### Upsampling & downsampling

In [39]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

In [40]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345678)
    
    return features_downsampled, target_downsampled

In [41]:
features_upsampled, target_upsampled = upsample(features_train, target_train, 2)
features_sampled, target_sampled = downsample(features_upsampled, target_upsampled, 0.7)

features_upsampled_2, target_upsampled_2 = upsample(features_train_2, target_train_2, 2)
features_sampled_2, target_sampled_2 = downsample(features_upsampled_2, target_upsampled_2, 0.7)

print(target_sampled[target_sampled==1].count() / target_sampled.count())
print(target_sampled_2[target_sampled_2==1].count() / target_sampled_2.count())

0.4222636815920398
0.422191814902351


In [42]:
features_sampled.shape

(6432, 11)

In [43]:
features_sampled_2.shape

(8039, 11)

#### Случайный лес

In [44]:
%%time
forest_model = RandomForestClassifier(random_state=12345, class_weight='balanced_subsample')

params = dict(n_estimators=range(115,141,2), max_depth=range(14,19))
grid = GridSearchCV(forest_model, params, n_jobs=4, verbose=0, scoring='f1', cv=2)
grid.fit(features_sampled_2, target_sampled_2);

print(grid.best_params_)
best_forest = grid.best_estimator_
print(best_forest)

predicted_sampled_2 = best_forest.predict(features_sampled_2)
print('f1 на увеличенной балансированной (обучающе-валидационной) выборке:', 
      f1_score(target_sampled_2, predicted_sampled_2))
# predicted_train = best_forest.predict(features_train)
# print('f1 на обучающей выборке:', f1_score(target_train, predicted_train))
# predicted_valid = best_forest.predict(features_valid)
# print('f1 на валидационной выборке:', f1_score(target_valid, predicted_valid))
predicted_test = best_forest.predict(features_test)
print('f1 на тестовой выборке:', f1_score(target_test, predicted_test))

{'max_depth': 16, 'n_estimators': 127}
RandomForestClassifier(bootstrap=True, class_weight='balanced_subsample',
                       criterion='gini', max_depth=16, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=127, n_jobs=None, oob_score=False,
                       random_state=12345, verbose=0, warm_start=False)
f1 на увеличенной балансированной (обучающе-валидационной) выборке: 0.998235294117647
f1 на тестовой выборке: 0.6291486291486292
Wall time: 1min 45s


In [45]:
probabilities_test = best_forest.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]
auc_roc = roc_auc_score(target_test, probabilities_one_test)
print('AUC-ROC на тестовой выборке:', auc_roc)

AUC-ROC на тестовой выборке: 0.8519792543995743


#### Логистическая регрессия

In [46]:
%%time
log_reg_model = LogisticRegression(random_state=1234567, max_iter=300, class_weight='balanced')

params = dict(C=[1,2,5,10], 
              penalty=['l2','l1'],
              solver=['liblinear','warn']
             ) 
grid = GridSearchCV(log_reg_model, params, verbose=0, scoring='f1', cv=2)
grid.fit(features_sampled_2, target_sampled_2);

print(grid.best_params_)
best_log_reg_model = grid.best_estimator_
print(best_log_reg_model)

predicted_sampled_2 = best_log_reg_model.predict(features_sampled_2)
print('f1 на увеличенной балансированной (обучающе-валидационной) выборке:', 
      f1_score(target_sampled_2, predicted_sampled_2))
# predicted_train = best_log_reg_model.predict(features_train)
# print('f1 на обучающей выборке:', f1_score(target_train, predicted_train))
# predicted_valid = best_log_reg_model.predict(features_valid)
# print("f1 на валидационной выборке:", f1_score(target_valid, predicted_valid))
predicted_test = best_log_reg_model.predict(features_test)
print('f1 на тестовой выборке:', f1_score(target_test, predicted_test))

{'C': 1, 'penalty': 'l1', 'solver': 'liblinear'}
LogisticRegression(C=1, class_weight='balanced', dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=300,
                   multi_class='warn', n_jobs=None, penalty='l1',
                   random_state=1234567, solver='liblinear', tol=0.0001,
                   verbose=0, warm_start=False)
f1 на увеличенной балансированной (обучающе-валидационной) выборке: 0.6646833829154272
f1 на тестовой выборке: 0.4964249233912155
Wall time: 1.3 s


In [47]:
probabilities_test = best_log_reg_model.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]
auc_roc = roc_auc_score(target_test, probabilities_one_test)
print('AUC-ROC на тестовой выборке:', auc_roc)

AUC-ROC на тестовой выборке: 0.7694778137328782


Выигрыш по сравнению с вариантом без апсэмплинга, даунсэмплинга крайне незначительный.

#### Метод k ближайших соседей

In [48]:
%%time
k_model = KNeighborsClassifier(n_jobs=4, weights='distance')

params = dict(n_neighbors=range(27,33,2), 
              #weights=['uniform','distance'], 
              #algorithm=['auto','ball_tree','kd_tree','brute'],
              p=[1,2],
              metric=['minkowski','euclidean','manhattan'],  #,'chebyshev'],
              leaf_size=range(1,3)
             )
grid = GridSearchCV(k_model, params, n_jobs=4, verbose=0, scoring='f1', cv=2)
grid.fit(features_sampled_2, target_sampled_2);

print(grid.best_params_)
best_k_model = grid.best_estimator_
print(best_k_model)

predicted_sampled_2 = best_k_model.predict(features_sampled_2)
print('f1 на увеличенной балансированной (обучающе-валидационной) выборке:', 
      f1_score(target_sampled_2, predicted_sampled_2))
# predicted_train = best_k_model.predict(features_train)
# print('f1 на обучающей выборке:', f1_score(target_train, predicted_train))
# predicted_valid = best_k_model.predict(features_valid)
# print('f1 на валидационной выборке:', f1_score(target_valid, predicted_valid))
predicted_test = best_k_model.predict(features_test)
print('f1 на тестовой выборке:', f1_score(target_test, predicted_test))

{'leaf_size': 1, 'metric': 'minkowski', 'n_neighbors': 31, 'p': 2}
KNeighborsClassifier(algorithm='auto', leaf_size=1, metric='minkowski',
                     metric_params=None, n_jobs=4, n_neighbors=31, p=2,
                     weights='distance')
f1 на увеличенной балансированной (обучающе-валидационной) выборке: 1.0
f1 на тестовой выборке: 0.5761006289308177
Wall time: 32.5 s


In [49]:
probabilities_test = best_k_model.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]
auc_roc = roc_auc_score(target_test, probabilities_one_test)
print('AUC-ROC на тестовой выборке:', auc_roc)

AUC-ROC на тестовой выборке: 0.8313710714127399


## Тестирование моделей

In [50]:
predicted_test = best_forest.predict(features_test)
probabilities_test = best_forest.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]
print(f"Случайный лес:\t\tf1: {f1_score(target_test, predicted_test):1.6f},",
      f"\tAUC-ROC: {roc_auc_score(target_test, probabilities_one_test):1.6f}")

predicted_test = best_log_reg_model.predict(features_test)
probabilities_test = best_log_reg_model.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]
print(f"Логистич.регрессия:\tf1: {f1_score(target_test, predicted_test):1.6f},",
      f"\tAUC-ROC: {roc_auc_score(target_test, probabilities_one_test):1.6f}")

predicted_test = best_k_model.predict(features_test)
probabilities_test = best_k_model.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]
print(f"k ближайших соседей:\tf1: {f1_score(target_test, predicted_test):1.6f},",
      f"\tAUC-ROC: {roc_auc_score(target_test, probabilities_one_test):1.6f}")

Случайный лес:		f1: 0.629149, 	AUC-ROC: 0.851979
Логистич.регрессия:	f1: 0.496425, 	AUC-ROC: 0.769478
k ближайших соседей:	f1: 0.576101, 	AUC-ROC: 0.831371


## Вывод

Было обучено 3 разных модели, с учётом и без учёта дисбаланса классов. Получить удовлетворительные результаты меры ***f1*** удалось только для случайного леса с учётом дисбаланса классов (2 способами). Лучший результат: **f1 ≈ 0.63**.

Но учёт дисбаланса довольно значительно повысил результаты и для случайного леса, и для логистической регрессии.

Если сравнивать **f1** и **AUC-ROC**, можно заметить следующее:
* AUC-ROC всегда больше, чем f1;
* f1 изменяется быстрее;
* между ними нет линейной зависимости (даже в рамках одного вида моделей), т.к. были получены результаты:

<br> f1: 0.286334,   AUC-ROC: 0.768279
<br> f1: 0.494382,   AUC-ROC: 0.769901
<br> f1: 0.496425,   AUC-ROC: 0.769478 ↓
<br> f1: 0.560886,   AUC-ROC: 0.850611
<br> f1: 0.576101, 	AUC-ROC: 0.831371 ↓
<br> f1: 0.610429,   AUC-ROC: 0.849953
<br> f1: 0.629149, 	 AUC-ROC: 0.851979

In [51]:
dt.datetime.today().strftime("%d.%m.%Y %H:%M")

'09.06.2022 13:58'